More information can be found at the link below
https://github.com/explodinggradients/ragas/blob/main/docs/getstarted/rag_testset_generation.md

# Setup

In [ ]:
# %pip install -r requirements.txt

# Environment Variables

In [2]:
# Get environment variables

from dotenv import load_dotenv


# Load environment variables from .env file
load_dotenv()

True

# Sample Docs

In [5]:
from langchain_community.document_loaders import DirectoryLoader

path = "docs/sample/"

loader = DirectoryLoader(path, 
                         glob="**/*.md", 
                         show_progress=True, 
                         use_multithreading=True)

docs = loader.load()


libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
100%|██████████| 11/11 [01:38<00:00,  8.99s/it]
Need to load profiles.
Need to load profiles.
Need to load profiles.
Need to load profiles.
Need to load profiles.
Need to load profiles.


Need to load profiles.
  9%|▉         | 1/11 [00:00<00:08,  1.16it/s]Need to load profiles.libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but a

In [6]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [ ]:
import time

from ragas.testset import TestsetGenerator
from ragas.testset.synthesizers import default_query_distribution

TESTSET_SIZE = 5

query_distribution = default_query_distribution(generator_llm)

# Create the generator 
generator = TestsetGenerator(llm=generator_llm, 
                             embedding_model=generator_embeddings)

# Generate the dataset
dataset = generator.generate_with_langchain_docs(docs, 
                                                 testset_size=TESTSET_SIZE,
                                                 query_distribution = query_distribution)


# Write the testet to disk
# Write the results to disk
timestr = time.strftime("%Y%m%d%H%M%S")
file_name = "sample_testset.csv_" + timestr 
dataset.to_csv((f"{file_name}"))

# Display the dataset
dataset.to_pandas()


Applying HeadlinesExtractor:   0%|          | 0/5 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/11 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/6 [00:00<?, ?it/s]

Property 'summary' already exists in node 'c3af03'. Skipping!


...

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/6 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,How does GitLab promote diversity and inclusio...,"[title: ""Building an Inclusive Remote Culture""...",GitLab promotes diversity and inclusion by ena...,single_hop_specifc_query_synthesizer
1,Wht is the significnce of Dversity in the cont...,"[Creating a Diversity, Inclusion & Belonging A...",The significance of diversity in the context o...,single_hop_specifc_query_synthesizer
2,What are the goals of the Sales Sponsorship Pi...,"[<1-hop>\n\ntitle: ""Sales Sponsorship Pilot Pr...",The goals of the Sales Sponsorship Pilot Progr...,multi_hop_abstract_query_synthesizer
3,What are the goals and benefits of the Sales S...,"[<1-hop>\n\ntitle: ""Sales Sponsorship Pilot Pr...",The goals of the Sales Sponsorship Pilot Progr...,multi_hop_abstract_query_synthesizer
4,How do DIB roundtables contribute to fostering...,[<1-hop>\n\nTips for Managers Set aside time t...,DIB roundtables contribute to fostering a sens...,multi_hop_specific_query_synthesizer
5,How do the practices of allyship discussed in ...,[<1-hop>\n\nTips on being an ally Identifying ...,The practices of allyship outlined in the tips...,multi_hop_specific_query_synthesizer


# 10-K Docs

In [9]:
from langchain_community.document_loaders import DirectoryLoader

path = "docs/10k/html"

loader = DirectoryLoader(path=path, 
                         glob="**/*.html", 
                         show_progress=True, 
                         use_multithreading=True)

docs = loader.load()

  0%|          | 0/2 [00:00<?, ?it/s]libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
100%|██████████| 2/2 [00:01<00:00,  1.22it/s]


In [ ]:
from ragas.testset import TestsetGenerator
from ragas.testset.synthesizers import default_query_distribution

TESTSET_SIZE = 50

query_distribution = default_query_distribution(generator_llm)

# Create the generator 
generator = TestsetGenerator(llm=generator_llm, 
                             embedding_model=generator_embeddings)

# Generate the dataset
dataset = generator.generate_with_langchain_docs(docs, 
                                                 testset_size=TESTSET_SIZE,
                                                 query_distribution = query_distribution)


# Write the testet to disk
timestr = time.strftime("%Y%m%d%H%M%S")
file_name = "10k_testset.csv_" + timestr
dataset.to_csv((f"{file_name}"))

# Display the dataset
dataset.to_pandas()

In [ ]:
testset_link = dataset.upload()
print(testset_link)